In [1]:
import ast
import glob
import json
import os
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output
from torchvision.ops import box_iou

In [2]:
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"
SEED = 37

In [3]:
df = pd.read_csv('/app/_data/train_alb.csv')

In [4]:
df

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,img_path,len_annotation,new_img_path,label,width_max,width_min,height_max,height_min,width_mean,height_mean,bbox
0,0,40258,0,0,0_0,[],/app/_data/tensorflow-great-barrier-reef/train...,0,/app/_data/images/0_0.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,40258,1,1,0_1,[],/app/_data/tensorflow-great-barrier-reef/train...,0,/app/_data/images/0_1.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,40258,2,2,0_2,[],/app/_data/tensorflow-great-barrier-reef/train...,0,/app/_data/images/0_2.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,40258,3,3,0_3,[],/app/_data/tensorflow-great-barrier-reef/train...,0,/app/_data/images/0_3.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,40258,4,4,0_4,[],/app/_data/tensorflow-great-barrier-reef/train...,0,/app/_data/images/0_4.jpg,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38230,2,29859,10631,2859,2_10631_1,[],/app/_data/tensorflow-great-barrier-reef/train...,1,/app/_data/augmented/images/2_10631_1.jpg,1,44.0,44.0,37.0,37.0,44.0,37.0,"[[51, 643, 44, 37]]"
38231,2,29859,10631,2859,2_10631_2,[],/app/_data/tensorflow-great-barrier-reef/train...,1,/app/_data/augmented/images/2_10631_2.jpg,1,44.0,44.0,37.0,37.0,44.0,37.0,"[[51, 643, 44, 37]]"
38232,2,29859,10632,2860,2_10632_0,[],/app/_data/tensorflow-great-barrier-reef/train...,1,/app/_data/augmented/images/2_10632_0.jpg,1,45.0,45.0,37.0,37.0,45.0,37.0,"[[38, 681, 45, 37]]"
38233,2,29859,10632,2860,2_10632_1,[],/app/_data/tensorflow-great-barrier-reef/train...,1,/app/_data/augmented/images/2_10632_1.jpg,1,45.0,45.0,37.0,37.0,45.0,37.0,"[[38, 681, 45, 37]]"


In [5]:
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)

In [6]:
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()

155

In [7]:
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

154

## KFold split

In [8]:
# kf = GroupKFold(n_splits=4)
# list_train_ids = []
# list_val_ids= []
# for train_idx, val_idx in (
#     kf.split(train_df, y=train_df.len_annotation, groups=train_df.sequence)
# ):
#     list_train_ids.append(train_idx)
#     list_val_ids.append(val_idx)
#     print(train_df.loc[val_idx, "len_annotation"].values.sum(), train_df.loc[train_idx, "len_annotation"].values.sum())

In [9]:
VIDEO_ID = 2
train = pd.concat(
    [
        df.query("video_id!=@VIDEO_ID and len_annotation!=0"),
        df.query("video_id!=@VIDEO_ID and len_annotation==0").sample(
            int(df.query("video_id!=@VIDEO_ID and len_annotation!=0").shape[0] * 0.03)
        ),
    ]
).sample(frac=1)
val = df.query("video_id==@VIDEO_ID").sample(frac=1)
train[["len_annotation", "label"]].sum()
val[["len_annotation", "label"]].sum()
train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()

len_annotation    37735
label             16949
dtype: int64

len_annotation    9781
label             2704
dtype: int64

len_annotation    3.857990
label             6.268121
dtype: float64

In [10]:
# df_split = (
#     df.groupby("subsequence_id")
#     .agg({"label": "max", "len_annotation": "sum", "video_frame": "count"})
#     .astype(int)
#     .reset_index()
# )
# n_splits = 10
# y=df_split["label"]
# skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

# for fold_id, (train_idx, val_idx) in enumerate(
#     skf.split(df_split["subsequence_id"], y=y)
# ):
#     subseq_val_idx = df_split["subsequence_id"].iloc[val_idx]
#     df.loc[df["subsequence_id"].isin(subseq_val_idx), "fold"] = fold_id

# df["fold"] = df["fold"].astype(int)
# for fold in range(10):
#     print(f"\nFold {fold}")
#     df.query("fold != @fold")[["len_annotation", "label"]].sum() / df.query(
#         "fold == @fold"
#     )[["len_annotation", "label"]].sum()

In [11]:
# KFOLD = 3
# train = pd.concat(
#     [
#         df.query("fold != @KFOLD and len_annotation!=0"),
#         df.query("fold != @KFOLD and len_annotation==0").sample(
#             int(df.query("fold != @KFOLD and len_annotation!=0").shape[0] * 0.07)
#         ),
#     ]
# ).sample(frac=1)
# val = df.query("fold == @KFOLD").sample(frac=1)
# train[["len_annotation", "label"]].sum()
# val[["len_annotation", "label"]].sum()
# train[["len_annotation", "label"]].sum() / val[["len_annotation", "label"]].sum()


In [12]:
train_ids = train.index.tolist()
val_ids = val.index.tolist()

In [13]:
train_txt =  f"/app/_data/train_aug_{SEED}_val{VIDEO_ID}.txt"
val_txt = train_txt.replace('train_', 'val_')
data_yaml_path = f'/app/_data/yolov5_f2/data/reef_data_aug_val{VIDEO_ID}.yaml'
data_yaml_path
val_txt

'/app/_data/yolov5_f2/data/reef_data_aug_val2.yaml'

'/app/_data/val_aug_37_val2.txt'

In [14]:
train_img_path = df.loc[train_ids, "new_img_path"].tolist()
val_img_path = df.loc[val_ids, "new_img_path"].tolist()
np.savetxt(
    train_txt,
    train_img_path,
    fmt="%s",
)
np.savetxt(val_txt, val_img_path, fmt="%s")

## Custimize parameters

In [15]:
from IPython.core.magic import register_line_cell_magic


@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, "w") as f:
        f.write(cell.format(**globals()))

In [16]:
%%writetemplate {data_yaml_path}

train: {train_txt} # training directory
val: {val_txt} # validation directory

# Classes
nc: 1  # number of classes
names: ['starfish']  # class names

In [17]:
!cat {data_yaml_path}


train: /app/_data/train_aug_37_val2.txt # training directory
val: /app/_data/val_aug_37_val2.txt # validation directory

# Classes
nc: 1  # number of classes
names: ['starfish']  # class names


In [18]:
!cat /app/_data/yolov5_f2/data/hyps/hyp.scratch.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (effic

In [19]:
%%writetemplate /app/_data/yolov5_f2/data/hyps/hyp.aug_custom.yaml
# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for COCO training from scratch
# python train.py --batch 40 --cfg yolov5m.yaml --weights '' --data coco.yaml --img 640 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.005  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.3  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.3  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.1  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.1  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)

In [20]:
!pip install --upgrade wandb
clear_output()
import wandb

wandb.login()

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)


True

In [21]:
%cd /app/_data/yolov5_f2/
!pip install -r requirements.txt
clear_output()

# Train

In [22]:
WEIGHTS = "yolov5l6.pt"
IMG_SIZE = 2560
NAME = f"{WEIGHTS[:-3]}_{IMG_SIZE}_val{VIDEO_ID}_rect_aug_f2"
NAME

'yolov5l6_2560_val2_rect_aug_f2'

In [23]:
# for path in glob.glob('/app/_data/yolov5_f2/runs/train/*rect_aug_f2*/'):
#     shutil.rmtree(path)

In [24]:
# shutil.rmtree('/app/_data/yolov5_f2/runs/train/NAME')

In [25]:
!python train.py --img {IMG_SIZE} \
                --batch 4\
                --epochs 80 \
                --data {data_yaml_path} \
                --weights {WEIGHTS} \
                --name {NAME} \
                --hyp data/hyps/hyp.aug_custom.yaml \
                --single-cls \
                --patience 10 \
                --rect \
                --workers 0

wandb: Currently logged in as: tatanko (use `wandb login --relogin` to force relogin)
train: weights=yolov5l6.pt, cfg=, data=/app/_data/yolov5_f2/data/reef_data_aug_val2.yaml, hyp=data/hyps/hyp.aug_custom.yaml, epochs=80, batch_size=4, imgsz=2560, rect=True, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=True, optimizer=SGD, sync_bn=False, workers=0, project=runs/train, name=yolov5l6_2560_val2_rect_aug_f2, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=10, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (Docker image), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

hyperparameters: lr0=0.005, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_mom

In [15]:
glob.glob("/app/_data/yolov5_f2/runs/train/*/weights/*.pt")

['/app/_data/yolov5/runs/train/yolov5m6_val2/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5m6_val2/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5l6_val110/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5l6_val110/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5s6_val29/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5s6_val29/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5l6_val2/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5l6_val2/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5m_val1_1/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5m_val1_1/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5m6_val23/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5m6_val23/weights/best.pt',
 '/app/_data/yolov5/runs/train/yolov5m6_val1/weights/last.pt',
 '/app/_data/yolov5/runs/train/yolov5m6_val1/weights/best.pt']

In [20]:
# for path in glob.glob('/app/_data/yolov5_f2/runs/train/*/'):
#     shutil.rmtree(path)

In [22]:
!cp -R /app/_data/yolov5 /app/_data/YOLOv5_kaggle/yolov5

In [25]:
shutil.rmtree("/app/_data/YOLOv5_kaggle/yolov5_f2/.ipynb_checkpoints")

In [19]:
shutil.copy(
    "/app/_data/yolov5_f2/runs/train/yolov5l6_val110/weights/best.pt",
    "/app/_data/YOLOv5Weights/yolov5l6_val110.pt",
)

'/app/_data/YOLOv5Weights/yolov5l6_val110.pt'

In [ ]:
shutil.copy("/app/_data/YOLOv5Weights")

In [26]:
!ls -a /app/_data/YOLOv5_kaggle/yolov5

.		.gitignore		 data		   setup.cfg
..		.pre-commit-config.yaml  detect.py	   train.py
.dockerignore	CONTRIBUTING.md		 export.py	   tutorial.ipynb
.git		Dockerfile		 hubconf.py	   utils
.gitattributes	LICENSE			 models		   val.py
.github		README.md		 requirements.txt


In [23]:
def tp_fp_fn(gt, prediction, conf_thr):
    ious = np.arange(0.3, 0.81, 0.05)
    TP, FP, FN = (
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
    )
    prediction = prediction[prediction[:, 4] > conf_thr]
    bboxes = prediction[:, :4].astype("int")
    bboxes[:, 0] = bboxes[:, 0] - bboxes[:, 2] / 2
    bboxes[:, 1] = bboxes[:, 1] - bboxes[:, 3] / 2
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    if bboxes.size != 0:
        if gt.size == 0:
            fp = bboxes.shape[0]
            FP = np.full(ious.shape[0], fp, "int16")
        else:
            iou_matrix = box_iou(torch.Tensor(gt), torch.Tensor(bboxes))
            for n, iou_thr in enumerate(ious):
                x = torch.where(iou_matrix >= iou_thr)
                tp = np.unique(x[0]).shape[0]
                fp = bboxes.shape[0] - tp
                fn = gt.shape[0] - tp
                TP[n] = tp
                FP[n] = fp
                FN[n] = fn
    else:
        if gt.size != 0:
            fn = gt.shape[0]
            FN = np.full(ious.shape[0], fn, "int16")
    return TP, FP, FN

In [24]:
with open("/app/f2_results.json", "r") as f:
    res_dict = json.load(f)

In [35]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)

In [36]:
val = df[df['video_id']==2].reset_index(drop=True)

In [33]:
df_test["annotations"] = df_test["annotations"].apply(lambda x: ast.literal_eval(x))

In [37]:
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])

path = f'/app/_data/yolov5_f2/runs/train/{NAME}/weights/best.pt'
IMG_SIZE = IMG_SIZE
model = torch.hub.load(
    "/app/_data/yolov5_f2", "custom", path=path, source="local", force_reload=True
)
model.conf = 0.01
# chose validation set
df_test = val.copy()
# computing f2 score
for ix in tqdm(df_test.index.tolist()):
    img = np.array(Image.open(df_test.loc[ix, "img_path"]))
    prediction = model(img, size=IMG_SIZE, augment=True).xywh[0].cpu().numpy()
    prediction = prediction[prediction[:, 4] > 0.1]
    gt = np.array([list(x.values()) for x in df_test.loc[ix, "annotations"]])
    if gt.size:
        gt[:, 2] = gt[:, 2] + gt[:, 0]
        gt[:, 3] = gt[:, 3] + gt[:, 1]
    for n, c_th in enumerate(conf_thres):
        TP, FP, FN = tp_fp_fn(gt, prediction, c_th)
        res[n, 0, :] += TP
        res[n, 1, :] += FP
        res[n, 2, :] += FN

YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients, 110.0 GFLOPs
Adding AutoShape... 
100% 8561/8561 [28:05<00:00,  5.08it/s]


In [38]:
F2 = np.zeros(conf_thres.shape[0])
for c in range(conf_thres.shape[0]):
    TP = res[c, 0, :]
    FP = res[c, 1, :]
    FN = res[c, 2, :]
    recall = TP / (TP + FN)
    precission = TP / (TP + FP)
    f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
    F2[c] = np.mean(f2)
if path not in res_dict:
    res_dict[path] = {
        IMG_SIZE: {
            "best": [
                np.round(conf_thres[np.argmax(F2)], 2),
                np.round(np.max(F2), 4),
            ],
            "all": list(np.round(F2, 4)),
        }
    }
else:
    res_dict[path][IMG_SIZE] = {
        "best": [
            np.round(conf_thres[np.argmax(F2)], 2),
            np.round(np.max(F2), 4),
        ],
        "all": list(np.round(F2, 4)),
    }

In [39]:
res_dict[path][IMG_SIZE]

{'best': [0.12, 0.6357],
 'all': [0.6327,
  0.6346,
  0.6357,
  0.635,
  0.6344,
  0.6347,
  0.6339,
  0.6341,
  0.6327,
  0.6324,
  0.6304,
  0.6302,
  0.6295,
  0.629,
  0.6273,
  0.6256,
  0.6248,
  0.6246,
  0.6223,
  0.6206,
  0.6201,
  0.6196,
  0.6191,
  0.6179,
  0.6162,
  0.6156,
  0.6148,
  0.6147,
  0.6132,
  0.6127,
  0.612,
  0.6094,
  0.6075,
  0.6066,
  0.6044,
  0.6027,
  0.6022,
  0.6013,
  0.5993,
  0.5983,
  0.5972,
  0.5955,
  0.594,
  0.5928,
  0.5911,
  0.5892,
  0.5884,
  0.5852,
  0.5805,
  0.5791,
  0.5747]}

In [ ]:
with open("/app/f2_results.json", "w") as f:
    json.dump(res_dict, f)